## Convolutional DNN

In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from attack_utils import get_synonymes#, attack, predict_sentence
import random
import nltk
import operator
from tqdm import tqdm

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rusha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\rusha\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rusha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
MAX_SEQUENCE_LENGTH = 5000
MAX_NUM_WORDS = 25000
EMBEDDING_DIM = 300
TEST_SPLIT = 0.2
SAVE_MODEL_PATH = './save/model/'
TEXT_DATA = 'data/fake_or_real_news.csv'

In [3]:
# read in our data and preprocess it

df = pd.read_csv(TEXT_DATA)
df.drop(labels=['id','title'], axis='columns', inplace=True)
# only select stories with lengths gt 0 -- there are some texts with len = 0
mask = list(df['text'].apply(lambda x: len(x) > 0))
df = df[mask]

In [4]:
# prepare text samples and their labels

texts = df['text']
labels = df['label']

print('Found %s texts.' %texts.shape[0])

Found 6335 texts.


In [5]:
# vectorize the text samples into a 2D integer tensor 
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
data = pad_sequences(sequences, 
                     maxlen=MAX_SEQUENCE_LENGTH, 
                     padding='pre', 
                     truncating='pre')

print('Found %s unique tokens.' % len(word_index))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Using TensorFlow backend.


Found 99682 unique tokens.
Shape of data tensor: (6335, 5000)
Shape of label tensor: (6335,)


In [6]:
# split the data into a training set and a validation set   

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(data, 
                                                  labels.apply(lambda x: 0 if x == 'FAKE' else 1), 
                                                  test_size=TEST_SPLIT)

In [7]:
# build a 1D convnet with global maxpooling                                                                      

from keras import layers
from keras.models import Sequential

model = Sequential(
    [
        # part 1: word and sequence processing
        layers.Embedding(num_words,
                         EMBEDDING_DIM, 
                         input_length=MAX_SEQUENCE_LENGTH,
                         trainable=True),
        layers.Conv1D(128, 5, activation='relu'),
        layers.GlobalMaxPooling1D(),
        
        # part 2: classification
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 5000, 300)         7500300   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 4996, 128)         192128    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 7,709,069
Trainable params: 7,709,069
Non-trainable params: 0
_________________________________________________________________


In [8]:
# define a function that allows us to evaluate our models

from sklearn.metrics import accuracy_score

def evaluate_model(predict_fun, X_train, y_train, X_test, y_test):
    '''
    evaluate the model, both training and testing errors are reported
    '''
    # training error
    y_predict_train = predict_fun(X_train)
    train_acc = accuracy_score(y_train,y_predict_train)
    
    # testing error
    y_predict_test = predict_fun(X_test)
    test_acc = accuracy_score(y_test,y_predict_test)
    
    return train_acc, test_acc

In [9]:
# define a function that allows us to evaluate our models

from sklearn.metrics import accuracy_score

def evaluate_ourfakes(model, Fake_texts):
    '''
    evaluate the model, both training and testing errors are reported
    '''
    missclasified_fakes = []
    correct_clasified_fakes = []
    def predict(X):
        return np.rint(model.predict(X))
    # training error
    y_predict = predict(Fake_texts)
    y = np.zeros(len(y_predict))
    for i in range(len(y_predict)):
        missclasified_fakes.append(Fake_texts[i]) if y_predict[i] == 1 else correct_clasified_fakes.append(Fake_texts[i])
    acc = accuracy_score(y,y_predict) #model.evaluate(Fake_texts,y)
    return acc, missclasified_fakes, correct_clasified_fakes

In [11]:
model.load_weights(SAVE_MODEL_PATH+'CNN_model.h5') # replace X with the last saved checkpoint number


In [12]:
def predict_sentence(model,sent):
    try_vector = tokenizer.texts_to_sequences([sent])
    try_vector = pad_sequences([try_vector[0]], 
                     maxlen=MAX_SEQUENCE_LENGTH, 
                     padding='pre', 
                     truncating='pre')
    val = model.predict(try_vector)
    return val


In [13]:
faker = list(df['label'].apply(lambda x: x == 'FAKE'))
fakes = df[faker]
fake_texts = fakes['text']
# fake_labels = fakes['label']

In [14]:
fake_text_vectors = tokenizer.texts_to_sequences(fake_texts)
fake_text_vectors = pad_sequences(fake_text_vectors, 
                     maxlen=MAX_SEQUENCE_LENGTH, 
                     padding='pre', 
                     truncating='pre')

In [15]:
acc, missclasified_fakes, correct_clasified_fakes = evaluate_ourfakes(model, fake_text_vectors)

In [16]:
print("total missclassified fakes",len(missclasified_fakes),"total correctly classified fakes",len(correct_clasified_fakes))

total missclassified fakes 2 total correctly classified fakes 3162


In [17]:
correct_fakes =tokenizer.sequences_to_texts(correct_clasified_fakes)

In [18]:
# with open(f'CNN1d_correct_fakes.txt','w', encoding="utf-8") as f:
#     f.writelines(correct_fakes)

In [19]:
# with open(f'CNN1d_correct_fakes.txt','r', encoding="utf-8") as f:
#     correct_fakes = f.readlines()

In [20]:
def attack(dummy,pertub=1, printSwaps = False):    
    dummy_temp = nltk.word_tokenize(dummy[:MAX_SEQUENCE_LENGTH])
    done = False
    unsuccessfullSwaps = 0
    for i in range(int(len(dummy_temp)*pertub)):
        if done:
            break
        itter = 0
        while itter<10000:
            itter+=1
            v = random.randint(0,len(dummy_temp)-1)
            syns,_ = get_synonymes(dummy_temp[v])
            if len(syns)>0:
                break
        if itter == 10000:
            print("didnt catch any synonymes")
            continue
        candidates = {}
        # candidate is a scored candidate dictionary storing all the 
        # synonymes with prediction Score
        word = dummy_temp[v]
        sent = " ".join(dummy_temp)
        candidates[word]=predict_sentence(model,sent)
        for s in syns:
            dummy_temp[v] = s
            sent = (" ".join(dummy_temp))
            val = predict_sentence(model,sent)
            candidates[s] = val
            if val>=0.5:
#                 print("got changes in ",i,val)
                done = True
                break
    #         best_candidate = max(candidates.iteritems(), key=operator.itemgetter(1))[0]
        best_candidate = max(candidates, key=candidates.get)
        if word == best_candidate:
            unsuccessfullSwaps+=1
#             if printSwaps:
#                 print("did not swap")
#         else:
#             if printSwaps:
#             print(word,best_candidate)
        dummy_temp[v] = best_candidate
    if not done:
#         print("sorry")
        return 0,None
    print("swaps done ",i-unsuccessfullSwaps,"total words",len(dummy_temp),"unsucessfull swaps try",unsuccessfullSwaps)
    return i-unsuccessfullSwaps," ".join(dummy_temp)

In [39]:
success_on = np.zeros(len(correct_fakes),dtype='int16')
# for i in tqdm(range(len(correct_fakes))):
# for i in tqdm(range(300)):
for i in tqdm(range(200)):
    success_on[i],_ = attack(correct_fakes[i],1)









  0%|          | 0/299 [00:00<?, ?it/s]







  0%|          | 1/299 [01:47<8:54:14, 107.57s/it]

swaps done  67 total words 455 unsucessfull swaps try 176










  1%|          | 2/299 [03:54<9:20:35, 113.25s/it]

swaps done  85 total words 427 unsucessfull swaps try 199










  1%|          | 3/299 [09:51<15:19:46, 186.44s/it]

...

swaps done  125 total words 677 unsucessfull swaps try 452










 68%|██████▊   | 203/299 [10:22:59<5:31:22, 207.11s/it]

swaps done  72 total words 895 unsucessfull swaps try 410










 68%|██████▊   | 204/299 [10:27:58<6:11:33, 234.67s/it]







 69%|██████▊   | 205/299 [10:31:01<5:43:07, 219.02s/it]

KeyboardInterrupt: 

In [21]:
success_on.dump(SAVE_MODEL_PATH+'CNN_success_rate.npy')

In [43]:
sum(success_on[:200]>0)

85

In [21]:
attack(correct_fakes[0],1)

swaps done  121 total words 919 unsucessfull swaps try 473


(120,
 'think greenfield a journalism fellow at the unsusceptibility center is a new york writer focusing on radical islam in the final move of the vote hillary rodham clinton has gone to operation with the fbi the son “ unprecedented ” give birth been engage around so often this election that it ought to be displace but information technology ’ s wind unprecedented for the nominee of metric linear unit commissioned military officer political party to go war with the fbi but that ’ s exactly what hillary and her be have done patients just waking up now and watching an hour of cnn from their hospital beds would assume that fbi supervisor crowd comey is hillary ’ s adversary in this election the federal bureau of investigation exist under activity by everyone from obama to cnn hillary ’ chemical element multitude have circulated a letter attacking comey there equal currently more occupation hit pieces lambasting him than targeting trump it wouldn ’ nucleotide equal too surprising if the 